In [15]:
#### -----------------------------------------------------------------------------------------------------------------------------------------
#### author: Ranjan Barman, date: May 5, 2025
#### Mapped PathAI HIFs to TCGA biomarker status with Age
#### ------------------------------------------------------------------------------------------------------------------------------------------
import os, sys, pickle, bz2
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from time import time 
from tqdm import tqdm

_wpath_ = "/data/Lab_ruppin/Ranjan/HnE/"        # set working directory as the parent directory where all datasets are saved
os.chdir(_wpath_)

print(f"working directory = {_wpath_}\n")

working directory = /data/Lab_ruppin/Ranjan/HnE/



In [16]:
# Define dataset and file paths
dataset_name = "TCGA_BRCA_FFPE"
outcome_names = ["HER2_Status", "PR_Status", "ER_Status"]


# File paths
PathAI_HIFs_with_BRCA_status_file = "PA_HIF_BRCA/clinical_tcga_brca_outputs/tcga_brca_subtype_to_original_hifs.csv"

pathAI_TCGA_BRCA_survival_data_file = "PA_NUHIF_BRCA/data/PathAI_BRCA_MetaData.xlsx"

out_path = f"{dataset_name}/outputs/PathAI/Survival_V2/"

os.makedirs(out_path, exist_ok=True) # Creates directory if it doesn't exist already

# Load data
data_full = pd.read_csv(PathAI_HIFs_with_BRCA_status_file)
patient_list = data_full['sample_id'].reset_index(drop=True)
data_full
data_full


# Filter to include only rows where outcome is "Positive" or "Negative"
data_filtered = data_full[data_full[outcome_names].isin(["Positive", "Negative"]).all(axis=1)].copy() # Use .copy() to avoid SettingWithCopyWarning
filtered_patient_list = patient_list[data_filtered.index]

# Create a new DataFrame for original nuHIFs with proper status
data_orginal_HIFs = data_filtered.copy()  # Keep a copy of the filtered data before dropping columns

data_orginal_HIFs




# The number of samples used for TCGA BRCA subtype prediction 
TCGA_BRCA_Subtypes_556_file = f"{dataset_name}/outputs/HoverNet/Subtypes/outputs_biomarker_status_prediction_results_All_HoverNet_NPIFs/combined_class_predictions_all_features_for_subtypes.csv"

TCGA_BRCA_Subtypes_556 = pd.read_csv(TCGA_BRCA_Subtypes_556_file)

# Convert the first column ("sample") to a series
sample_list_556 = TCGA_BRCA_Subtypes_556['sample_id'].reset_index(drop=True)

# Filter sample based on `sample_list_556`
PathAI_HIFs_with_BRCA_status = data_orginal_HIFs[data_orginal_HIFs['sample_id'].isin(sample_list_556)].copy()
PathAI_HIFs_with_BRCA_status

#read files 
pathAI_TCGA_BRCA_survival_data = pd.read_excel(pathAI_TCGA_BRCA_survival_data_file)



# Rename the columns in pathAI_TCGA_BRCA_survival_data
pathAI_TCGA_BRCA_survival_data = pathAI_TCGA_BRCA_survival_data.rename(columns={"age_at_initial_pathologic_diagnosis": "Age", "ajcc_pathologic_tumor_stage": "Stage"})

PathAI_HIFs_with_BRCA_status
# pathAI_TCGA_BRCA_survival_data



,sample_id,HER2_Status,PR_Status,ER_Status,AREA (MM2) OF [EPITHELIAL] IN [TISSUE]_HE,AREA (MM2) OF [ESI_0080] IN [TISSUE]_HE,AREA (MM2) OF [NECROSIS] IN [TISSUE]_HE,AREA (MM2) OF [STROMA] IN [TISSUE]_HE,AREA (MM2) OF [TUMOR] IN [TISSUE]_HE,AREA PROP [[EPITHELIAL] OVER [TUMOR]] IN [TISSUE]_HE,...,TOTAL [LYMPHOCYTE CELLS] IN [STROMA]_HE,TOTAL [LYMPHOCYTE CELLS] IN [TUMOR]_HE,TOTAL [MACROPHAGE CELLS] IN [EPITHELIAL]_HE,TOTAL [MACROPHAGE CELLS] IN [ESI_0080]_HE,TOTAL [MACROPHAGE CELLS] IN [STROMA]_HE,TOTAL [MACROPHAGE CELLS] IN [TUMOR]_HE,TOTAL [PLASMA CELLS] IN [EPITHELIAL]_HE,TOTAL [PLASMA CELLS] IN [ESI_0080]_HE,TOTAL [PLASMA CELLS] IN [STROMA]_HE,TOTAL [PLASMA CELLS] IN [TUMOR]_HE
0,TCGA-A1-A0SB,Negative,Negative,Positive,5.884160,8.505324,1.389632,12.406016,18.290176,0.321712,...,19166.0,24610.0,706.0,1362.0,2353.0,3059.0,825.0,949.0,1383.0,2208.0
1,TCGA-A1-A0SD,Negative,Positive,Positive,7.959952,15.879072,1.843632,18.089680,26.049632,0.305569,...,8826.0,10351.0,999.0,1207.0,2520.0,3519.0,311.0,412.0,523.0,834.0
2,TCGA-A1-A0SE,Negative,Positive,Positive,10.708352,20.749744,0.011680,17.366224,28.074576,0.381425,...,2739.0,3675.0,146.0,412.0,465.0,611.0,144.0,269.0,260.0,404.0
3,TCGA-A1-A0SF,Negative,Positive,Positive,1.600576,13.634196,0.061824,31.309600,32.910176,0.048635,...,37415.0,39063.0,1171.0,4474.0,8911.0,10082.0,252.0,1546.0,3023.0,3275.0
4,TCGA-A1-A0SH,Negative,Positive,Negative,6.305520,14.302604,0.235024,13.158768,19.464288,0.323953,...,5098.0,5776.0,310.0,442.0,936.0,1246.0,207.0,389.0,427.0,634.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,TCGA-EW-A1PB,Negative,Negative,Negative,103.655120,92.218780,12.361056,41.098096,144.753216,0.716082,...,97532.0,167641.0,4818.0,17008.0,20185.0,25003.0,8546.0,14439.0,10629.0,19175.0
719,TCGA-EW-A1PD,Positive,Positive,Positive,12.700832,22.796892,0.269024,24.320192,37.021024,0.343071,...,7446.0,8802.0,859.0,1119.0,3124.0,3983.0,215.0,261.0,526.0,741.0
720,TCGA-EW-A1PE,Negative,Positive,Positive,87.381168,109.834856,1.369792,73.197504,160.578672,0.544164,...,32669.0,42024.0,2351.0,5264.0,7560.0,9911.0,3636.0,7865.0,7104.0,10740.0
721,TCGA-EW-A1PF,Negative,Positive,Positive,15.787328,39.763816,0.046576,34.822016,50.609344,0.311945,...,37521.0,40134.0,946.0,3141.0,6808.0,7754.0,453.0,1587.0,2399.0,2852.0


In [17]:
# Define the desired order for the specified columns
desired_columns = ['os', 'os_time', 'pfs', 'pfs_time', 'Age', 'Stage', 'bcr_patient_barcode']

# Apply the new column order to the DataFrame
pathAI_TCGA_BRCA_survival_data_with_desired_columns = pathAI_TCGA_BRCA_survival_data[desired_columns]
pathAI_TCGA_BRCA_survival_data_with_desired_columns


,os,os_time,pfs,pfs_time,Age,Stage,bcr_patient_barcode
0,0,8,0,8,30,Stage IIB,TCGA-PL-A8LY
1,1,2009,1,2009,79,Stage I,TCGA-BH-A18S
2,0,446,0,446,62,Stage IIB,TCGA-A7-A4SC
3,0,433,0,433,71,Stage IIA,TCGA-D8-A1Y2
4,0,385,0,385,55,Stage IIA,TCGA-C8-A12V
...,...,...,...,...,...,...,...
881,0,1150,0,1150,54,Stage IIB,TCGA-AR-A1AS
882,0,317,0,317,38,Stage IIIA,TCGA-C8-A1HL
883,0,849,1,681,73,Stage IIIA,TCGA-LQ-A4E4
884,0,172,0,172,32,Stage IA,TCGA-AN-A0FF


In [18]:
# 'sample_id' is the column name in df1 and 'Sample' is the column name in df2
# Adjust these column names to match the actual column names in your CSV files
merged_df1 = pd.merge(PathAI_HIFs_with_BRCA_status, pathAI_TCGA_BRCA_survival_data_with_desired_columns, left_on='sample_id', right_on='bcr_patient_barcode', how='inner')

# Drop the redundant 'bcr_patient_barcode' column
merged_df1.drop(columns='bcr_patient_barcode', inplace=True)

merged_df1

,sample_id,HER2_Status,PR_Status,ER_Status,AREA (MM2) OF [EPITHELIAL] IN [TISSUE]_HE,AREA (MM2) OF [ESI_0080] IN [TISSUE]_HE,AREA (MM2) OF [NECROSIS] IN [TISSUE]_HE,AREA (MM2) OF [STROMA] IN [TISSUE]_HE,AREA (MM2) OF [TUMOR] IN [TISSUE]_HE,AREA PROP [[EPITHELIAL] OVER [TUMOR]] IN [TISSUE]_HE,...,TOTAL [PLASMA CELLS] IN [EPITHELIAL]_HE,TOTAL [PLASMA CELLS] IN [ESI_0080]_HE,TOTAL [PLASMA CELLS] IN [STROMA]_HE,TOTAL [PLASMA CELLS] IN [TUMOR]_HE,os,os_time,pfs,pfs_time,Age,Stage
0,TCGA-A1-A0SB,Negative,Negative,Positive,5.884160,8.505324,1.389632,12.406016,18.290176,0.321712,...,825.0,949.0,1383.0,2208.0,0,259,0,259,70,Stage I
1,TCGA-A1-A0SD,Negative,Positive,Positive,7.959952,15.879072,1.843632,18.089680,26.049632,0.305569,...,311.0,412.0,523.0,834.0,0,437,0,437,59,Stage IIA
2,TCGA-A1-A0SE,Negative,Positive,Positive,10.708352,20.749744,0.011680,17.366224,28.074576,0.381425,...,144.0,269.0,260.0,404.0,0,1321,0,1321,56,Stage I
3,TCGA-A1-A0SF,Negative,Positive,Positive,1.600576,13.634196,0.061824,31.309600,32.910176,0.048635,...,252.0,1546.0,3023.0,3275.0,0,1463,0,1463,54,Stage IIA
4,TCGA-A1-A0SH,Negative,Positive,Negative,6.305520,14.302604,0.235024,13.158768,19.464288,0.323953,...,207.0,389.0,427.0,634.0,0,1437,0,1437,39,Stage IIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,TCGA-EW-A1PB,Negative,Negative,Negative,103.655120,92.218780,12.361056,41.098096,144.753216,0.716082,...,8546.0,14439.0,10629.0,19175.0,0,608,0,608,70,Stage IIIA
552,TCGA-EW-A1PD,Positive,Positive,Positive,12.700832,22.796892,0.269024,24.320192,37.021024,0.343071,...,215.0,261.0,526.0,741.0,0,424,0,424,61,Stage IIA
553,TCGA-EW-A1PE,Negative,Positive,Positive,87.381168,109.834856,1.369792,73.197504,160.578672,0.544164,...,3636.0,7865.0,7104.0,10740.0,0,320,0,320,56,Stage IIA
554,TCGA-EW-A1PF,Negative,Positive,Positive,15.787328,39.763816,0.046576,34.822016,50.609344,0.311945,...,453.0,1587.0,2399.0,2852.0,0,439,0,439,50,Stage IA


In [19]:
# Path for the output CSV file
BrcaBiomarkerStatusToPathAI_HIFsToSurvival = f"{out_path}BrcaBiomarkerStatusToPathAI_HIFsToSurvival.csv"

# Write the merged dataframe to a new CSV file
merged_df1.to_csv(BrcaBiomarkerStatusToPathAI_HIFsToSurvival, index=False)

print("The files have been mapped and saved to:", BrcaBiomarkerStatusToPathAI_HIFsToSurvival)
print("Done!")

The files have been mapped and saved to: TCGA_BRCA_FFPE/outputs/PathAI/Survival_V2/BrcaBiomarkerStatusToPathAI_HIFsToSurvival.csv
Done!
